In [ ]:
!pip install "numpy<1.24"
!git clone https://github.com/ifzhang/ByteTrack.git
!cd ByteTrack && pip3 install -r requirements.txt
!cd ByteTrack && python3 setup.py develop
!pip install cython_bbox --quiet
!pip install ultralytics --quiet
!pip install onemetric --quiet
!pip install loguru
!pip install lap
!pip install motmetrics
!pip install ultralytics cython_bbox loguru onnxruntime lap motmetrics
!git clone https://github.com/ifzhang/ByteTrack.git
%cd ByteTrack
!pip install -r requirements.txt
!pip install -e .
!pip install kaggle
import kagglehub
path = kagglehub.dataset_download("ayushspai/sportsmot")
!cp -r /root/.cache/kagglehub/datasets/ayushspai/sportsmot/versions/1/sportsmot_publish/splits_txt /content/drive/MyDrive/Project/

Cloning into 'ByteTrack'...
remote: Enumerating objects: 2007, done.
remote: Total 2007 (delta 0), reused 0 (delta 0), pack-reused 2007 (from 1)
Receiving objects: 100% (2007/2007), 79.60 MiB | 19.76 MiB/s, done.
Resolving deltas: 100% (1141/1141), done.
Updating files: 100% (229/229), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 12.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 67.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
ERROR: Could not find a version that satisfies the requirement onnxruntime==1.8.0 (from versions: 1.15.0, 1.15.1, 1.16.0, 1.16.1, 1.16.2, 1.16.3, 1.17.0, 1.17.1, 1.17.3, 1.18.0, 1.18.1, 1.19.0, 1.19.2, 1.20.0, 1.20.1)
ERROR: No matching distribution found for onnxruntime==1.8.0
running develop
/usr/local/lib/python3.11/dist-

100%|██████████| 33.5G/33.5G [05:44<00:00, 105MB/s]

Extracting files...


cp: cannot create directory '/content/drive/MyDrive/Project/': No such file or directory


In [ ]:
from google.colab import files
import shutil
import cv2
import os
import torch
import numpy as np
from ultralytics import YOLO
from yolox.tracking_utils.timer import Timer
from types import SimpleNamespace
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from yolox.tracker.byte_tracker import BYTETracker
from yolox.tracking_utils.timer import Timer
import argparse

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
uploaded = files.upload()
os.makedirs("/content/models", exist_ok=True)
shutil.move("weights.pt", "/content/models/weights.pt")

Saving weights.pt to weights.pt


'/content/models/weights.pt'

In [ ]:
from google.colab import files
uploaded = files.upload()  # Choose 2.mp4 from your local machine


Saving 2.mp4 to 2.mp4


In [ ]:
from pathlib import Path
import time as time
import tqdm as tqdm
# Define the paths for model and video input/output
model_path = Path("/content/models/weights.pt")
input_video_path =  Path("2.mp4")

output_video_path = Path("/content/mot_output.mp4")

# Initialize YOLO model for detection
detector = YOLO(str(model_path))

# Set parameters for BYTETracker
tracker_params = SimpleNamespace(
    track_thresh=0.5,    # Detection threshold for tracking
    track_buffer=30,     # Number of frames to retain a track without detection
    mot20=False,         # MOT20 dataset setting (False by default)
    match_thresh=0.8     # Matching threshold for assignment process
)

# Initialize BYTETracker
tracker = BYTETracker(tracker_params, frame_rate=25)

# Open the video capture
video_capture = cv2.VideoCapture(str(input_video_path))
if not video_capture.isOpened():
    raise IOError(f"Unable to open video file: {input_video_path}")

# Video properties
frame_width = int(video_capture.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = 25  # Frames per second of the input video

# Initialize the video writer for saving the output video
video_writer = cv2.VideoWriter(str(output_video_path), cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width, frame_height))

frame_counter = 0  # To count frames
total_processing_time = 0  # Track the total processing time

# Timer class to measure execution time
class Timer:
    def __init__(self):
        self.start_time = None
        self.end_time = None

    def tic(self):
        """Start the timer."""
        self.start_time = time.time()

    def toc(self):
        """Stop the timer and return the elapsed time."""
        self.end_time = time.time()
        return self.end_time - self.start_time

# Full list of player names for all players (Player A, Player B, ...)
player_names = {i: f"Player {chr(65 + i)}" for i in range(30)}  # Creates Player A, Player B, ..., Player AD

# Using tqdm to show progress bar for video processing
for success, current_frame in tqdm.tqdm(iterable=iter(lambda: video_capture.read(), (False, None)), total=int(video_capture.get(cv2.CAP_PROP_FRAME_COUNT))):
    if not success:
        break

    frame_counter += 1
    start_time = time.time()

    # Perform object detection using YOLO
    detection_results = detector(current_frame)[0]

    # List for bounding boxes (x1, y1, x2, y2, confidence, class_id)
    detection_boxes = []

    # Process detections
    for detection in detection_results.boxes:
        x1, y1, x2, y2 = detection.xyxy[0].tolist()
        confidence = float(detection.conf[0])
        detection_boxes.append([x1, y1, x2, y2, confidence, 1.0])  # class_id is set as 1.0 for all

    # Convert detection results to tensor format
    detection_tensor = torch.tensor(detection_boxes, dtype=torch.float32) if detection_boxes else torch.empty((0, 6), dtype=torch.float32)

    # Timer for frame processing time calculation
    frame_timer = Timer()
    frame_timer.tic()

    # Update the tracker with new detections
    online_tracks = tracker.update(detection_tensor, (frame_height, frame_width), (frame_height, frame_width))

    frame_timer.toc()

    # Calculate the processing time for the current frame
    frame_processing_time = time.time() - start_time
    total_processing_time += frame_processing_time

    # Add metrics and labels on the frame
    for track in online_tracks:
        track_id = track.track_id
        top_left_x, top_left_y, bottom_right_x, bottom_right_y = map(int, track.tlbr)  # Get tracked bounding box coordinates

        # Use a single color for all bounding boxes and paths (green)
        color = (0, 255, 0)

        # Draw bounding box with thicker lines for better visibility
        cv2.rectangle(current_frame, (top_left_x, top_left_y), (bottom_right_x, bottom_right_y), color, 4)

        # Add player name based on track_id (use the player_names dictionary)
        player_name = player_names.get(track_id, f"Player {track_id}")

        # Add the player name label with a semi-transparent background for better visibility
        label_background = current_frame[top_left_y-30:top_left_y, top_left_x:bottom_right_x]
        label_background = cv2.addWeighted(label_background, 0.5, label_background, 0.5, 0)
        cv2.putText(current_frame, player_name, (top_left_x, top_left_y - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 1.0, (255, 255, 255), 2, cv2.LINE_AA)

        # Add a track path (lines)
        if hasattr(track, 'track_history') and len(track.track_history) > 1:
            for i in range(1, len(track.track_history)):
                cv2.line(current_frame, track.track_history[i - 1], track.track_history[i], color, 3)

    # Add average processing time to the top-left corner
    avg_frame_time = total_processing_time / frame_counter if frame_counter > 0 else 0
    cv2.putText(current_frame, f"Avg Frame Time: {avg_frame_time:.3f}s", (10, 40),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

    # Add FPS to the top-left corner
    cv2.putText(current_frame, f"FPS: {fps:.2f}", (10, 70),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

    # Write the frame to output video file
    video_writer.write(current_frame)

# Release resources after processing
video_capture.release()
video_writer.release()

# Inform the user of completion with performance metrics
print(f"Tracking completed successfully. The output video has been saved at {output_video_path}.")
print(f"Total frames processed: {frame_counter}")
print(f"Average frame processing time: {total_processing_time / frame_counter:.3f} seconds")


  0%|          | 0/901 [00:00<?, ?it/s]


0: 544x960 12 players, 46.7ms
Speed: 19.1ms preprocess, 46.7ms inference, 353.2ms postprocess per image at shape (1, 3, 544, 960)


  0%|          | 1/901 [00:02<42:04,  2.80s/it]


0: 544x960 12 players, 45.8ms
Speed: 7.5ms preprocess, 45.8ms inference, 2.0ms postprocess per image at shape (1, 3, 544, 960)


  0%|          | 2/901 [00:03<19:28,  1.30s/it]

...


0: 544x960 1 ball, 12 players, 44.5ms
Speed: 5.7ms preprocess, 44.5ms inference, 1.4ms postprocess per image at shape (1, 3, 544, 960)


100%|█████████▉| 900/901 [02:31<00:00,  6.42it/s]


0: 544x960 11 players, 44.5ms
Speed: 7.7ms preprocess, 44.5ms inference, 1.6ms postprocess per image at shape (1, 3, 544, 960)


100%|██████████| 901/901 [02:31<00:00,  5.93it/s]

Tracking completed successfully. The output video has been saved at /content/mot_output.mp4.
Total frames processed: 901
Average frame processing time: 0.071 seconds


In [ ]:
from google.colab import files
files.download("/content/mot_output.mp4")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>